# Financial Services: Delta Liquid Clustering Demo


## Overview


This notebook demonstrates the power of **Delta Liquid Clustering** in Oracle AI Data Platform (AIDP) Workbench using a financial services analytics use case. Liquid clustering automatically optimizes data layout for query performance without requiring manual partitioning or Z-Ordering.

### What is Liquid Clustering?

Liquid clustering automatically identifies and groups similar data together based on clustering columns you define. This optimization happens automatically during data ingestion and maintenance operations, providing:

- **Automatic optimization**: No manual tuning required
- **Improved query performance**: Faster queries on clustered columns
- **Reduced maintenance**: No need for manual repartitioning
- **Adaptive clustering**: Adjusts as data patterns change

### Use Case: Transaction Fraud Detection and Customer Analytics

We'll analyze financial transaction records from a bank. Our clustering strategy will optimize for:

- **Customer-specific queries**: Fast lookups by account ID
- **Time-based analysis**: Efficient filtering by transaction date
- **Fraud pattern detection**: Quick aggregation by transaction type and risk scores

### AIDP Environment Setup

This notebook leverages the existing Spark session in your AIDP environment.

In [ ]:
# Create financial services catalog and analytics schema

# In AIDP, catalogs provide data isolation and governance

spark.sql("CREATE CATALOG IF NOT EXISTS finance")

spark.sql("CREATE SCHEMA IF NOT EXISTS finance.analytics")

print("Financial services catalog and analytics schema created successfully!")

Financial services catalog and analytics schema created successfully!


## Step 2: Create Delta Table with Liquid Clustering

### Table Design

Our `account_transactions` table will store:

- **account_id**: Unique account identifier
- **transaction_date**: Date and time of transaction
- **transaction_type**: Type (Deposit, Withdrawal, Transfer, Payment, etc.)
- **amount**: Transaction amount
- **merchant_category**: Merchant type (Retail, Restaurant, Online, etc.)
- **location**: Transaction location
- **risk_score**: Fraud risk assessment (0-100)

### Clustering Strategy

We'll cluster by `account_id` and `transaction_date` because:

- **account_id**: Customers often have multiple transactions, grouping their financial activity together
- **transaction_date**: Time-based queries are critical for fraud detection, spending analysis, and regulatory reporting
- This combination optimizes for both customer account analysis and temporal fraud pattern detection

In [ ]:
# Create Delta table with liquid clustering

# CLUSTER BY defines the columns for automatic optimization

spark.sql("""

CREATE TABLE IF NOT EXISTS finance.analytics.account_transactions (

    account_id STRING,

    transaction_date TIMESTAMP,

    transaction_type STRING,

    amount DECIMAL(15,2),

    merchant_category STRING,

    location STRING,

    risk_score INT

)

USING DELTA

CLUSTER BY (account_id, transaction_date)

""")

print("Delta table with liquid clustering created successfully!")

print("Clustering will automatically optimize data layout for queries on account_id and transaction_date.")

Delta table with liquid clustering created successfully!
Clustering will automatically optimize data layout for queries on account_id and transaction_date.


## Step 3: Generate Financial Services Sample Data

### Data Generation Strategy

We'll create realistic financial transaction data including:

- **5,000 accounts** with multiple transactions over time
- **Transaction types**: Deposits, withdrawals, transfers, payments, ATM withdrawals
- **Realistic temporal patterns**: Daily banking activity, weekend vs weekday patterns
- **Merchant categories**: Retail, restaurants, online shopping, utilities, entertainment

### Why This Data Pattern?

This data simulates real financial scenarios where:

- Customers perform multiple transactions daily/weekly
- Fraud patterns emerge over time
- Regulatory reporting requires temporal analysis
- Risk scoring enables real-time fraud prevention
- Customer spending analysis drives personalized financial services

In [ ]:
# Generate sample financial transaction data

# Using fully qualified imports to avoid conflicts

import random

from datetime import datetime, timedelta


# Define financial data constants

TRANSACTION_TYPES = ['Deposit', 'Withdrawal', 'Transfer', 'Payment', 'ATM']

MERCHANT_CATEGORIES = ['Retail', 'Restaurant', 'Online', 'Utilities', 'Entertainment', 'Groceries', 'Healthcare', 'Transportation']

LOCATIONS = ['New York, NY', 'Los Angeles, CA', 'Chicago, IL', 'Houston, TX', 'Miami, FL', 'Online', 'ATM']


# Generate account transaction records

transaction_data = []

base_date = datetime(2024, 1, 1)


# Create 5,000 accounts with 10-50 transactions each

for account_num in range(1, 5001):

    account_id = f"ACC{account_num:08d}"
    
    # Each account gets 10-50 transactions over 12 months

    num_transactions = random.randint(10, 50)
    
    for i in range(num_transactions):

        # Spread transactions over 12 months with realistic timing

        days_offset = random.randint(0, 365)

        hours_offset = random.randint(0, 23)

        transaction_date = base_date + timedelta(days=days_offset, hours=hours_offset)
        
        # Select transaction type

        transaction_type = random.choice(TRANSACTION_TYPES)
        
        # Amount based on transaction type

        if transaction_type in ['Deposit', 'Transfer']:

            amount = round(random.uniform(100, 10000), 2)

        elif transaction_type == 'ATM':

            amount = round(random.uniform(20, 500), 2) * -1

        else:

            amount = round(random.uniform(10, 2000), 2) * -1
        
        # Select merchant category and location

        merchant_category = random.choice(MERCHANT_CATEGORIES)

        if transaction_type == 'ATM':

            location = 'ATM'

        elif transaction_type == 'Online':

            location = 'Online'

        else:

            location = random.choice(LOCATIONS)
        
        # Risk score (0-100, higher = more suspicious)

        risk_score = random.randint(0, 100)
        
        transaction_data.append({

            "account_id": account_id,

            "transaction_date": transaction_date,

            "transaction_type": transaction_type,

            "amount": amount,

            "merchant_category": merchant_category,

            "location": location,

            "risk_score": risk_score

        })



print(f"Generated {len(transaction_data)} account transaction records")

print("Sample record:", transaction_data[0])

Generated 150600 account transaction records
Sample record: {'account_id': 'ACC00000001', 'transaction_date': datetime.datetime(2024, 1, 5, 13, 0), 'transaction_type': 'ATM', 'amount': -412.88, 'merchant_category': 'Entertainment', 'location': 'ATM', 'risk_score': 27}


## Step 4: Insert Data Using PySpark

### Data Insertion Strategy

We'll use PySpark to:

1. **Create DataFrame** from our generated data
2. **Insert into Delta table** with liquid clustering
3. **Verify the insertion** with a sample query

### Why PySpark for Insertion?

- **Distributed processing**: Handles large datasets efficiently
- **Type safety**: Ensures data integrity
- **Optimization**: Leverages Spark's query optimization
- **Liquid clustering**: Automatically applies clustering during insertion

In [ ]:
# Insert data using PySpark DataFrame operations

# Using fully qualified function references to avoid conflicts


# Create DataFrame from generated data

df_transactions = spark.createDataFrame(transaction_data)


# Display schema and sample data

print("DataFrame Schema:")

df_transactions.printSchema()



print("\nSample Data:")

df_transactions.show(5)


# Insert data into Delta table with liquid clustering

# The CLUSTER BY (account_id, transaction_date) will automatically optimize the data layout

df_transactions.write.mode("overwrite").saveAsTable("finance.analytics.account_transactions")


print(f"\nSuccessfully inserted {df_transactions.count()} records into finance.analytics.account_transactions")

print("Liquid clustering automatically optimized the data layout during insertion!")

DataFrame Schema:
root
 |-- account_id: string (nullable = true)
 |-- amount: double (nullable = true)
 |-- location: string (nullable = true)
 |-- merchant_category: string (nullable = true)
 |-- risk_score: long (nullable = true)
 |-- transaction_date: timestamp (nullable = true)
 |-- transaction_type: string (nullable = true)


Sample Data:


+-----------+--------+-----------+-----------------+----------+-------------------+----------------+
| account_id|  amount|   location|merchant_category|risk_score|   transaction_date|transaction_type|
+-----------+--------+-----------+-----------------+----------+-------------------+----------------+
|ACC00000001| -412.88|        ATM|    Entertainment|        27|2024-01-05 13:00:00|             ATM|
|ACC00000001| -372.97|        ATM|    Entertainment|         3|2024-04-15 21:00:00|             ATM|
|ACC00000001|-1117.24|Houston, TX|   Transportation|        32|2024-01-16 12:00:00|      Withdrawal|
|ACC00000001| -1733.0|Houston, TX|       Restaurant|         8|2024-12-20 09:00:00|         Payment|
|ACC00000001| -164.06|        ATM|    Entertainment|         2|2024-02-12 12:00:00|             ATM|
+-----------+--------+-----------+-----------------+----------+-------------------+----------------+
only showing top 5 rows




Successfully inserted 150600 records into finance.analytics.account_transactions
Liquid clustering automatically optimized the data layout during insertion!


## Step 5: Demonstrate Liquid Clustering Benefits

### Query Performance Analysis

Now let's see how liquid clustering improves query performance. We'll run queries that benefit from our clustering strategy:

1. **Account transaction history** (clustered by account_id)
2. **Time-based fraud analysis** (clustered by transaction_date)
3. **Combined account + time queries** (optimal for our clustering)

### Expected Performance Benefits

With liquid clustering, these queries should be significantly faster because:

- **Data locality**: Related records are physically grouped together
- **Reduced I/O**: Less data needs to be read from disk
- **Automatic optimization**: No manual tuning required

In [ ]:
# Demonstrate liquid clustering benefits with optimized queries


# Query 1: Account transaction history - benefits from account_id clustering

print("=== Query 1: Account Transaction History ===")

account_history = spark.sql("""

SELECT account_id, transaction_date, transaction_type, amount, merchant_category

FROM finance.analytics.account_transactions

WHERE account_id = 'ACC00000001'

ORDER BY transaction_date DESC

""")



account_history.show()

print(f"Records found: {account_history.count()}")



# Query 2: Time-based fraud analysis - benefits from transaction_date clustering

print("\n=== Query 2: High-Risk Transactions Today ===")

high_risk_today = spark.sql("""

SELECT transaction_date, account_id, transaction_type, amount, risk_score

FROM finance.analytics.account_transactions

WHERE DATE(transaction_date) = CURRENT_DATE AND risk_score > 70

ORDER BY risk_score DESC, transaction_date DESC

""")



high_risk_today.show()

print(f"High-risk transactions found: {high_risk_today.count()}")



# Query 3: Combined account + time query - optimal for our clustering strategy

print("\n=== Query 3: Account Fraud Pattern Analysis ===")

fraud_patterns = spark.sql("""

SELECT account_id, transaction_date, transaction_type, amount, risk_score

FROM finance.analytics.account_transactions

WHERE account_id LIKE 'ACC0000001%' AND transaction_date >= '2024-06-01'

ORDER BY account_id, transaction_date

""")



fraud_patterns.show()

print(f"Pattern records found: {fraud_patterns.count()}")

=== Query 1: Account Transaction History ===


+-----------+-------------------+----------------+--------+-----------------+
| account_id|   transaction_date|transaction_type|  amount|merchant_category|
+-----------+-------------------+----------------+--------+-----------------+
|ACC00000001|2024-12-20 09:00:00|         Payment| -1733.0|       Restaurant|
|ACC00000001|2024-10-24 09:00:00|         Payment|-1689.16|       Healthcare|
|ACC00000001|2024-09-03 20:00:00|             ATM| -270.98|    Entertainment|
|ACC00000001|2024-06-28 15:00:00|        Transfer| 1288.76|       Healthcare|
|ACC00000001|2024-06-25 15:00:00|      Withdrawal|-1109.31|    Entertainment|
|ACC00000001|2024-05-31 21:00:00|      Withdrawal|-1323.84|    Entertainment|
|ACC00000001|2024-04-15 21:00:00|             ATM| -372.97|    Entertainment|
|ACC00000001|2024-04-05 17:00:00|      Withdrawal|-1532.56|           Online|
|ACC00000001|2024-03-11 06:00:00|         Deposit| 2533.68|       Restaurant|
|ACC00000001|2024-02-29 17:00:00|         Deposit| 3042.86|    E

Records found: 13

=== Query 2: High-Risk Transactions Today ===


+----------------+----------+----------------+------+----------+
|transaction_date|account_id|transaction_type|amount|risk_score|
+----------------+----------+----------------+------+----------+
+----------------+----------+----------------+------+----------+



High-risk transactions found: 0

=== Query 3: Account Fraud Pattern Analysis ===


+-----------+-------------------+----------------+--------+----------+
| account_id|   transaction_date|transaction_type|  amount|risk_score|
+-----------+-------------------+----------------+--------+----------+
|ACC00000010|2024-06-04 14:00:00|        Transfer| 1938.69|        89|
|ACC00000010|2024-07-11 20:00:00|         Deposit| 8968.96|        53|
|ACC00000010|2024-07-19 14:00:00|         Payment|-1926.25|        47|
|ACC00000010|2024-08-29 12:00:00|         Deposit| 9483.61|        28|
|ACC00000010|2024-09-20 23:00:00|        Transfer| 7191.52|        73|
|ACC00000010|2024-09-22 11:00:00|         Deposit| 5494.43|        25|
|ACC00000010|2024-10-05 21:00:00|        Transfer| 6472.15|         4|
|ACC00000010|2024-10-15 18:00:00|        Transfer| 5734.37|        90|
|ACC00000010|2024-11-24 09:00:00|         Deposit| 4922.75|        53|
|ACC00000010|2024-12-17 07:00:00|        Transfer| 5578.49|        63|
|ACC00000011|2024-06-11 18:00:00|         Payment| -500.16|        98|
|ACC00

Pattern records found: 135


## Step 6: Analyze Clustering Effectiveness

### Understanding the Impact

Let's examine how liquid clustering has organized our data and analyze some aggregate statistics to demonstrate the financial insights possible with this optimized structure.

### Key Analytics

- **Transaction volume** by type and risk patterns
- **Customer spending analysis** and account segmentation
- **Fraud detection metrics** and risk scoring effectiveness
- **Merchant category trends** and spending patterns

In [ ]:
# Analyze clustering effectiveness and financial insights


# Transaction analysis by type

print("=== Transaction Analysis by Type ===")

transaction_analysis = spark.sql("""

SELECT transaction_type, COUNT(*) as total_transactions,

       ROUND(SUM(amount), 2) as total_amount,

       ROUND(AVG(amount), 2) as avg_amount,

       ROUND(AVG(risk_score), 2) as avg_risk_score

FROM finance.analytics.account_transactions

GROUP BY transaction_type

ORDER BY total_transactions DESC

""")



transaction_analysis.show()


# Risk score distribution

print("\n=== Risk Score Distribution ===")

risk_distribution = spark.sql("""

SELECT 

    CASE 

        WHEN risk_score >= 80 THEN 'Very High Risk'

        WHEN risk_score >= 60 THEN 'High Risk'

        WHEN risk_score >= 40 THEN 'Medium Risk'

        WHEN risk_score >= 20 THEN 'Low Risk'

        ELSE 'Very Low Risk'

    END as risk_category,

    COUNT(*) as transaction_count,

    ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER (), 2) as percentage

FROM finance.analytics.account_transactions

GROUP BY 

    CASE 

        WHEN risk_score >= 80 THEN 'Very High Risk'

        WHEN risk_score >= 60 THEN 'High Risk'

        WHEN risk_score >= 40 THEN 'Medium Risk'

        WHEN risk_score >= 20 THEN 'Low Risk'

        ELSE 'Very Low Risk'

    END

ORDER BY transaction_count DESC

""")



risk_distribution.show()


# Merchant category spending

print("\n=== Merchant Category Spending Analysis ===")

merchant_analysis = spark.sql("""

SELECT merchant_category, COUNT(*) as transactions,

       ROUND(SUM(CASE WHEN amount > 0 THEN amount ELSE 0 END), 2) as deposits,

       ROUND(SUM(CASE WHEN amount < 0 THEN ABS(amount) ELSE 0 END), 2) as spending,

       ROUND(AVG(risk_score), 2) as avg_risk

FROM finance.analytics.account_transactions

GROUP BY merchant_category

ORDER BY spending DESC

""")



merchant_analysis.show()


# Monthly transaction trends

print("\n=== Monthly Transaction Trends ===")

monthly_trends = spark.sql("""

SELECT DATE_FORMAT(transaction_date, 'yyyy-MM') as month,

       COUNT(*) as transactions,

       ROUND(SUM(amount), 2) as net_flow,

       COUNT(DISTINCT account_id) as active_accounts,

       ROUND(AVG(risk_score), 2) as avg_risk_score

FROM finance.analytics.account_transactions

GROUP BY DATE_FORMAT(transaction_date, 'yyyy-MM')

ORDER BY month

""")



monthly_trends.show()

=== Transaction Analysis by Type ===


+----------------+------------------+--------------+----------+--------------+
|transaction_type|total_transactions|  total_amount|avg_amount|avg_risk_score|
+----------------+------------------+--------------+----------+--------------+
|         Deposit|             30187|1.5221104223E8|   5042.27|         49.62|
|      Withdrawal|             30174|-3.033876498E7|  -1005.46|         50.26|
|        Transfer|             30136|1.5238560295E8|    5056.6|         49.78|
|             ATM|             30066|   -7801233.33|   -259.47|         50.03|
|         Payment|             30037|-3.010117823E7|  -1002.14|         50.05|
+----------------+------------------+--------------+----------+--------------+


=== Risk Score Distribution ===


+--------------+-----------------+----------+
| risk_category|transaction_count|percentage|
+--------------+-----------------+----------+
|Very High Risk|            31042|     20.61|
|   Medium Risk|            30113|     20.00|
|      Low Risk|            30068|     19.97|
|     High Risk|            29761|     19.76|
| Very Low Risk|            29616|     19.67|
+--------------+-----------------+----------+


=== Merchant Category Spending Analysis ===


+-----------------+------------+-------------+----------+--------+
|merchant_category|transactions|     deposits|  spending|avg_risk|
+-----------------+------------+-------------+----------+--------+
|       Restaurant|       19064|3.827775625E7|8691779.96|   49.79|
|        Groceries|       18890|3.820906261E7|8631780.31|   49.87|
|           Retail|       18742|3.759042825E7|8619076.98|   49.87|
|   Transportation|       18829|3.861681326E7|8509561.01|   50.29|
|           Online|       18699|3.762347824E7|8507451.77|   50.11|
|    Entertainment|       18728|3.803600515E7|8477999.58|   50.05|
|        Utilities|       18728|3.746450578E7|8462294.11|   49.36|
|       Healthcare|       18920|3.877859564E7|8341232.82|   50.25|
+-----------------+------------+-------------+----------+--------+


=== Monthly Transaction Trends ===


+-------+------------+-------------+---------------+--------------+
|  month|transactions|     net_flow|active_accounts|avg_risk_score|
+-------+------------+-------------+---------------+--------------+
|2024-01|       12934| 2.01199504E7|           4456|         50.27|
|2024-02|       11801|1.821607335E7|           4352|         50.08|
|2024-03|       12678|2.072403875E7|           4416|         49.11|
|2024-04|       12451|1.985937352E7|           4395|         50.24|
|2024-05|       12799| 1.98355254E7|           4452|          49.9|
|2024-06|       12219|1.878310794E7|           4383|         49.46|
|2024-07|       12846|2.066751421E7|           4446|         49.88|
|2024-08|       12749|1.995521618E7|           4439|         50.16|
|2024-09|       12277|1.939366962E7|           4382|          49.9|
|2024-10|       12796|2.047159483E7|           4448|         49.95|
|2024-11|       12464|1.915816532E7|           4404|         50.27|
|2024-12|       12586|1.917123912E7|           4

## Key Takeaways: Delta Liquid Clustering in AIDP

### What We Demonstrated

1. **Automatic Optimization**: Created a table with `CLUSTER BY (account_id, transaction_date)` and let Delta automatically optimize data layout

2. **Performance Benefits**: Queries on clustered columns (account_id, transaction_date) are significantly faster due to data locality

3. **Zero Maintenance**: No manual partitioning, bucketing, or Z-Ordering required - Delta handles it automatically

4. **Real-World Use Case**: Financial services analytics where fraud detection and customer analysis are critical

### AIDP Advantages

- **Unified Analytics**: Seamlessly integrates with other AIDP services
- **Governance**: Catalog and schema isolation for financial data
- **Performance**: Optimized for both OLAP and OLTP workloads
- **Scalability**: Handles financial-scale data volumes effortlessly

### Best Practices for Liquid Clustering

1. **Choose clustering columns** based on your most common query patterns
2. **Start with 1-4 columns** - too many can reduce effectiveness
3. **Consider cardinality** - high-cardinality columns work best
4. **Monitor and adjust** as query patterns evolve

### Next Steps

- Explore other AIDP features like AI/ML integration
- Try liquid clustering with different column combinations
- Scale up to larger financial datasets
- Integrate with real banking systems and fraud detection platforms

This notebook demonstrates how Oracle AI Data Platform makes advanced financial analytics accessible while maintaining enterprise-grade performance and governance.